title: "평균 흐름 연습"
---
*작성자: 날씨 프로그램, 최종 업데이트 2024년 5월*

이 노트북은 평균 날씨 패턴 분석을 연습하기 위한 선택적 연습입니다. 다음 과정을 안내합니다:

1. Google Cloud에서 두 개의 다른 데이터 세트에 액세스합니다.
2. 두 데이터 세트 모두에서 기후학적 평균을 계산합니다.
3. 다른 압력 수준과 다른 데이터 세트에 대한 평균 흐름을 시각화합니다.

### 라이브러리 가져오기

In [ ]:
import xarray as xr
import gcsfs
import matplotlib.pyplot as plt

from dask.distributed import Client, progress

### Dask 클라이언트

Dask 클라이언트가 실행 중인 경우 계산이 훨씬 빨라집니다. 그렇지 않은 경우에도 계산은 실행되지만 시간이 더 걸립니다.

In [ ]:
client = Client()
client

### 1. Google Cloud에서 두 개의 다른 데이터 세트 액세스

두 가지 데이터 세트를 사용합니다:

*   **ERA5**: 유럽 중기 예보 센터(ECMWF) 재분석의 5세대. 이것은 1940년부터 현재까지 이어지는 고해상도(~31km) 재분석 데이터 세트입니다.
*   **GEFS**: 미국 국립 환경 예측 센터(NCEP)의 전지구 앙상블 예보 시스템. 이것은 하루에 네 번 실행되는 저해상도(~100km) 예보 모델입니다.

#### ERA5

먼저 ERA5 데이터에 액세스해 보겠습니다. 2023년 데이터를 사용합니다.

In [ ]:
gcs = gcsfs.GCSFileSystem(token='anon')
era5_path = 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-e5t.zarr-v1/'
era5_ds = xr.open_dataset(
    gcs.get_mapper(era5_path),
    chunks={'time': 48},
    engine='zarr',
)
era5_2023 = era5_ds.sel(time=slice('2023-01-01', '2023-12-31'))

데이터를 확인해 보겠습니다.

In [ ]:
era5_2023

#### GEFS

이제 GEFS 데이터에 액세스해 보겠습니다. 이것은 예보 모델이므로 ERA5 데이터와는 다른 차원을 가집니다. 예보의 리드 타임을 나타내는 `forecast_time` 차원과 앙상블의 다른 멤버를 나타내는 `ensemble_member` 차원이 있습니다.

In [ ]:
gefs_path = 'gcs://gcp-public-data-arco-gefs-v12/ar/1.0-degree/atmos/gefs_1p0_all_fhr.zarr/'
gefs_ds = xr.open_dataset(
    gcs.get_mapper(gefs_path),
    chunks={'time': 4},
    engine='zarr'
)

데이터를 확인해 보겠습니다.

In [ ]:
gefs_ds

### 2. 기후학적 평균 계산

이제 두 데이터 세트의 평균 흐름을 계산해 보겠습니다. ERA5 데이터의 경우 2023년 전체에 대한 평균을 계산합니다. GEFS 데이터의 경우 데이터 세트의 모든 예보에 대한 평균을 계산합니다.

#### ERA5

In [ ]:
era5_mean = era5_2023.mean('time').compute()

#### GEFS

GEFS 데이터의 경우 `time`, `forecast_time`, `ensemble_member` 차원에 대한 평균을 계산합니다.

In [ ]:
gefs_mean = gefs_ds.mean(['time', 'forecast_time', 'ensemble_member']).compute()

### 3. 평균 흐름 시각화

이제 평균 흐름을 시각화해 보겠습니다. 250hPa와 850hPa에서 바람의 평균 u-성분과 v-성분을 살펴보겠습니다. 250hPa 수준은 대류권 상단 근처이며 제트 기류가 위치한 곳입니다. 850hPa 수준은 대류권 하부에 있으며 종종 저고도 제트 및 수분 수송을 살펴보는 데 사용됩니다.

#### 250 hPa

먼저 두 데이터 세트에서 250hPa 수준을 선택합니다.

In [ ]:
era5_250 = era5_mean.sel(level=250)
gefs_250 = gefs_mean.sel(pressure_level=250)

이제 평균 풍속 벡터를 플로팅해 보겠습니다. `matplotlib.pyplot.quiver`를 사용하여 벡터를 플로팅합니다.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

# 플롯을 읽기 쉽게 만들기 위해 데이터 솎아내기
skip = 10

# ERA5
axes[0].quiver(
    era5_250['longitude'][::skip],
    era5_250['latitude'][::skip],
    era5_250['u_component_of_wind'][::skip, ::skip],
    era5_250['v_component_of_wind'][::skip, ::skip]
)
axes[0].set_title('ERA5 250 hPa 평균 풍속')
axes[0].set_xlabel('경도')
axes[0].set_ylabel('위도')

# GEFS
axes[1].quiver(
    gefs_250['longitude'][::skip],
    gefs_250['latitude'][::skip],
    gefs_250['u_component_of_wind'][::skip, ::skip],
    gefs_250['v_component_of_wind'][::skip, ::skip]
)
axes[1].set_title('GEFS 250 hPa 평균 풍속')
axes[1].set_xlabel('경도')

plt.suptitle('250 hPa 평균 풍속, 2023')
plt.tight_layout()
plt.show()

두 반구의 중위도에서 강한 제트 기류를 볼 수 있습니다. 북반구 제트가 더 강한데, 이는 북반구가 겨울이기 때문에 예상되는 현상입니다. 열대 지방에서는 동쪽에서 서쪽으로 부는 무역풍도 볼 수 있습니다.

#### 850 hPa

이제 850hPa 수준에 대해서도 동일한 작업을 수행합니다.

In [ ]:
era5_850 = era5_mean.sel(level=850)
gefs_850 = gefs_mean.sel(pressure_level=850)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)

# 플롯을 읽기 쉽게 만들기 위해 데이터 솎아내기
skip = 10

# ERA5
axes[0].quiver(
    era5_850['longitude'][::skip],
    era5_850['latitude'][::skip],
    era5_850['u_component_of_wind'][::skip, ::skip],
    era5_850['v_component_of_wind'][::skip, ::skip]
)
axes[0].set_title('ERA5 850 hPa 평균 풍속')
axes[0].set_xlabel('경도')
axes[0].set_ylabel('위도')

# GEFS
axes[1].quiver(
    gefs_850['longitude'][::skip],
    gefs_850['latitude'][::skip],
    gefs_850['u_component_of_wind'][::skip, ::skip],
    gefs_850['v_component_of_wind'][::skip, ::skip]
)
axes[1].set_title('GEFS 850 hPa 평균 풍속')
axes[1].set_xlabel('경도')

plt.suptitle('850 hPa 평균 풍속, 2023')
plt.tight_layout()
plt.show()

850hPa에서는 흐름이 250hPa보다 약합니다. 열대 지방에서는 여전히 무역풍을 볼 수 있지만 중위도 흐름은 그다지 조직적이지 않습니다. 또한 티베트 고원 주변의 흐름과 캘리포니아 해안의 흐름과 같은 몇 가지 흥미로운 특징도 볼 수 있습니다.

### 다음 단계

이 노트북은 평균 날씨 패턴 분석에 대한 간략한 소개였습니다. 이 분석을 확장할 수 있는 몇 가지 아이디어는 다음과 같습니다:

*   계절별 평균 흐름을 계산합니다.
*   GEFS 데이터에서 다양한 예보 리드 타임에 대한 평균 흐름을 계산합니다.
*   두 데이터 세트의 평균 흐름을 더 자세히 비교합니다. 예를 들어, 두 평균 흐름 간의 차이를 계산하거나 두 평균 흐름 간의 평균 제곱근 오차를 계산할 수 있습니다.
*   온도나 지오포텐셜 고도와 같은 다른 변수를 살펴봅니다.